In [41]:
# location of dataset folder
train_root = '/raid/binod/prashant/CVS_train'

import cv2
from PIL import Image

def extract_frame_by_index(video_path, frame_index, output_image_path=""):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Check if the video opened successfully
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return
    
    # Set the frame position
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
    
    # Read the frame
    ret, frame = cap.read()
    
    if ret:
        # # Save the frame as an image
        # cv2.imwrite(output_image_path, frame)
        # print(f"Frame {frame_index} extracted and saved to {output_image_path}")
        pass
    else:
        print(f"Error: Could not read frame {frame_index}")
    
    # Release the video capture object
    cap.release()
    img = cv2.cvtColor(cv2.resize(frame, (256, 256)), cv2.COLOR_BGR2RGB)
    return img

def extract_prev_frames_by_index(video_path, frame_index, output_image_path=""):
    # Open the video file
    imgs = []
    for _ in range(5):
        cap = cv2.VideoCapture(video_path)
        
        # Check if the video opened successfully
        if not cap.isOpened():
            print(f"Error: Could not open video {video_path}")
            return
        
        # Set the frame position
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        
        # Read the frame
        ret, frame = cap.read()
        
        if ret:
            # # Save the frame as an image
            # cv2.imwrite(output_image_path, frame)
            # print(f"Frame {frame_index} extracted and saved to {output_image_path}")
            pass
        else:
            print(f"Error: Could not read frame {frame_index}")
        
        # Release the video capture object
        cap.release()
        img = cv2.cvtColor(cv2.resize(frame, (256, 256)), cv2.COLOR_BGR2RGB)
        imgs.append(img)
        frame_index -= 30
        if frame_index < 0:
            break
    return imgs




In [42]:
import numpy as np
def find_mode(numbers_array):
    values, counts = np.unique(numbers_array, return_counts=True)
    
    # Find the index of the maximum count
    max_count_index = np.argmax(counts)
    
    # The mode is the value at the index of the maximum count
    mode = values[max_count_index]
    
    return mode

In [77]:
import os
import pandas as pd
import pickle
from tqdm import tqdm
all_lists = []
for video_id_mp4 in tqdm(os.listdir(os.path.join(train_root, 'videos'))):
    video_id = video_id_mp4[:-4]
    label_csv_path = os.path.join(train_root, 'labels', video_id, 'frame.csv')
    df  = pd.read_csv(label_csv_path)
    for index, row in df.iterrows():
        frame_index = row['frame_id']
        c1 = find_mode(np.array([row['c1_rater1'], row['c1_rater2'], row['c1_rater3']]))
        c2 = find_mode(np.array([row['c2_rater1'], row['c2_rater2'], row['c2_rater3']]))
        c3 = find_mode(np.array([row['c3_rater1'], row['c3_rater2'], row['c3_rater3']]))
        video_path = os.path.join(train_root, 'videos', f'{video_id}.mp4')
        img_array = extract_frame_by_index(video_path, frame_index)
        # img_arrays = extract_prev_frames_by_index(video_path, frame_index)

        dictionary = {
                        'video_id': video_id,
                        'frame_id': frame_index,
                        # 'img_arrays': img_arrays,
                        'img_array': img_array,
                        'label': [c1, c2, c3]
                    }
        all_lists.append(dictionary)
        # print(f"{frame_index}: ({c1}, {c2}, {c3})")

with open('single_images.pkl', 'wb') as handle:
# with open('multiple_images_1fps.pkl', 'wb') as handle:
    pickle.dump(all_lists, handle, protocol=pickle.HIGHEST_PROTOCOL)

  0%|                                                                                                                                        | 0/200 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [15:11<00:00,  4.56s/it]
